In [1]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from time import sleep

### Open Chrome

In [2]:
s = Service('/usr/local/bin/chromedriver')
driver = webdriver.Chrome(service=s)

### Access League Roster

In [3]:
driver.get('https://www.nba.com/players')
sleep(3)

In [4]:
toggle_switch = driver.find_element(By.CLASS_NAME, "PlayerList_toggle__RL_YD")
toggle_switch.click()
sleep(3)

### Get Table Headers

In [5]:
players_list = driver.find_element(By.CLASS_NAME, "players-list")
players_list_thead = players_list.find_element(By.TAG_NAME,'thead')
players_list_th = players_list_thead.find_element(By.TAG_NAME,'tr').find_elements(By.TAG_NAME,'th')
column_names = [th.text.title() for th in players_list_th]

### Get Players Bio

In [6]:
def next_page_exists():
    buttons = driver.find_elements(By.TAG_NAME, "button")
    for button in buttons:
        if button.get_attribute("title") == "Next Page Button" and not button.get_attribute("disabled"):
            return button
    return None

In [7]:
df = pd.DataFrame(columns=column_names)
    
page_count = 1
page_exists = True
while page_exists:
    print(f'Scanning page {page_count}')
    
    # Convert HTML table into DataFrame
    table_html = players_list.get_attribute('outerHTML')
    new_df = pd.read_html(table_html)[0]
    
    # Append the new DataFrame
    df = pd.concat([df,new_df], ignore_index=True)

    next_page_button = next_page_exists()
    
    if next_page_button:
        next_page_button.click()
        page_count += 1
    else:
        page_exists = False

Scanning page 1
Scanning page 2
Scanning page 3
Scanning page 4
Scanning page 5
Scanning page 6
Scanning page 7
Scanning page 8
Scanning page 9
Scanning page 10
Scanning page 11
Scanning page 12
Scanning page 13
Scanning page 14
Scanning page 15
Scanning page 16
Scanning page 17
Scanning page 18
Scanning page 19
Scanning page 20
Scanning page 21
Scanning page 22
Scanning page 23
Scanning page 24
Scanning page 25
Scanning page 26
Scanning page 27
Scanning page 28
Scanning page 29
Scanning page 30
Scanning page 31
Scanning page 32
Scanning page 33
Scanning page 34
Scanning page 35
Scanning page 36
Scanning page 37
Scanning page 38
Scanning page 39
Scanning page 40
Scanning page 41
Scanning page 42
Scanning page 43
Scanning page 44
Scanning page 45
Scanning page 46
Scanning page 47
Scanning page 48
Scanning page 49
Scanning page 50
Scanning page 51
Scanning page 52
Scanning page 53
Scanning page 54
Scanning page 55
Scanning page 56
Scanning page 57
Scanning page 58
Scanning page 59
Scanni

In [8]:
def insert_space(name_string):
    for i in range(1,len(name_string)):
        if name_string[i].isupper():
            return f"{name_string[:i]} {name_string[i:]}"

In [9]:
df["Player"] = df["Player"].apply(insert_space)

In [10]:
df.head()

,Player,Team,Number,Position,Height,Weight,Last Attended,Country
0,Alaa Abdelnaby,POR,30.0,F,6-10,240 lbs,Duke,USA
1,Zaid Abdul-Aziz,HOU,54.0,C,6-9,235 lbs,Iowa State,USA
2,Kareem Abdul-Jabbar,LAL,33.0,C,7-2,225 lbs,UCLA,USA
3,Mahmoud Abdul-Rauf,DEN,1.0,G,6-1,162 lbs,Louisiana State,USA
4,Tariq Abdul-Wahad,SAC,9.0,F-G,6-6,235 lbs,San Jose State,France


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4815 entries, 0 to 4814
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Player         4814 non-null   object
 1   Team           4815 non-null   object
 2   Number         4474 non-null   object
 3   Position       4765 non-null   object
 4   Height         4766 non-null   object
 5   Weight         4762 non-null   object
 6   Last Attended  4808 non-null   object
 7   Country        4813 non-null   object
dtypes: object(8)
memory usage: 301.1+ KB


In [12]:
df['Player'] = df['Player'].str.replace('\n',' ')
df['Weight'] = df['Weight'].str.replace(' lbs','')

In [13]:
df.to_csv('data_files/nba_league_roster_historical.csv', index=False)

In [14]:
driver.quit()